In [6]:
:opt no-lint

# Monad

## Outline

* Why `Monad`s?

* ...

* Showcasing the power of `Monad`
    * In a custom type
        * Implementing `Functor`, `Applicative`, and `Monad`
        * Using the power
    * Using someone else's code
        * Looking for `Monad` instances in Hackage
        * Using our knowledge 



Today, we will learn about the Monad type classe and how you can use it. This lesson assumes you saw and understood all the lessons up until now and that you did the homework. Especially lesson 11 about "Basic IO", lesson 18 about "Functor", and lesson 19 about "Applicative and Effects". This lesson is easy in comparison. So, if this lesson feels too hard, it's because you need to review those first.

## Why `Monad`s?

To understand why we need the `Monad` type class, we first need to see the limits of the `Appplicative` type class.

Let's say we are working on the backend of an app that requires user to sign up, and we have to validate the values somehow.

Here's the data type for a user:

In [1]:
-- Data type to represent a user
data User = User
  { userName :: String
  , userPassHash :: String
  , userEmail :: String
  } deriving (Show, Eq)

And here are a few dummy functions to validte the values:

In [2]:
import Data.Char (isAlphaNum, isDigit, isLower, isUpper, toLower)

normalizeUsername :: String -> Either String String
normalizeUsername uname
  | length uname < 3 = Left "Name must be at least 3 characters long"
  | all (\c -> isAlphaNum c || c == '_') uname = Right (map toLower uname)
  | otherwise = Left "Name must contain only alphanumeric characters and underscores"

checkAndGeneratePasswordHash :: String -> Either String String
checkAndGeneratePasswordHash password 
  | length password < 8 = Left "Password must be at least 8 characters long"
  | otherwise =
      if any isUpper password && any isLower password && any isDigit password
        then Right password -- Hashing logic
        else Left "Password must contain at least one uppercase, one lowercase, and one digit"

normalizeEmail :: String -> Either String String
normalizeEmail email
  | '@' `notElem` email = Left "Email must contain @ character"
  | otherwise = Right (map toLower email)

Of course, these are not functions you should use in production code. The important part is not how they are implemented, the *important part is how we use them*.

Now, let's create a new function that takes the username, password, and email and creates a user if everything is ok:

In [3]:
createUser :: String -> String -> String -> Either String User
createUser uname pass email =
  case normalizeUsername uname of
    Left err -> Left err
    Right normUname ->
      case checkAndGeneratePasswordHash pass of
        Left err -> Left err
        Right passHash ->
          case normalizeEmail email of
            Left err -> Left err
            Right normEmail ->
              Right (User normUname passHash normEmail)

As you can see, we have to deal with all the failure cases individually, and if everything is fine, we apply the `User` constructor to all our results at the end.

But wait a minute, we did a lot of work in the previous lesson to define the Applicative Functor type clas precisely so we can easily deal with this! Applicative allows us to apply a regular function (in this case the `User` type constructor) to three values at any Applycative level (in this case the `Either` Applicative level). So we can simplify the function by using the apply (`<*>`) operator like this:

In [4]:
createUser' :: String -> String -> String -> Either String User
createUser' uname pass email =
  User
    <$> normalizeUsername uname
    <*> checkAndGeneratePasswordHash pass
    <*> normalizeEmail email

There you go! Much better. Now, we can clearly see that we're creating a User by applying it's constructor to values that result from running the possibly failing `Either` effect of `normalizeUsername`, `checkAndGeneratePasswordHash`, and `normalizeEmail` functions.

We can run it to check if it works, of course:

In [11]:
-- Example usage:
createUser' "John" "paSs123456" "john@example.com"
createUser' ""     "pass123"    "john@example.com" 
createUser' "John" "john"       "john@example.com" 
createUser' "John" "Johnny1990" "john"            


(createUser "John" "paSs123456" "john@example.com") == (createUser' "John" "paSs123456" "john@example.com")

Right (User {userName = "john", userPassHash = "paSs123456", userEmail = "john@example.com"})

Left "Name must be at least 3 characters long"

Left "Password must be at least 8 characters long"

Left "Email must contain @ character"

True

We knew all this from the previous lesson. If this din't seem obvious to you, you might want to revie the Applictive Functor's lesson. 

If it made sense so far, lets make the code a little bit more realistic. Starting with changing the `User` type to this `UserAccount` one:

In [12]:
type UserId = String
type PasswordHash = String
type Email = String

data UserAccount = UserAccount
  { userId :: UserId
  , userName :: String
  , userPassHash :: PasswordHash
  , userEmail :: Email
  , userHasEnterprise :: Bool
  }
  deriving (Show, Eq)


As you can see, we use a few type synonyms to help with readibility and we change the fields to more realistic ones:

- We have `userId` that holds a unique id generated by our backend and that will never change.
- We have `userName` that holds the unique username chosen by the user and that it could be modified in the future.
- The `userPassHash` that holds the user's hashed password to verify its identity when they logg in.
- `userEmail` that holds the user's email.
- And `userHasEnterprise` which is a boolean flag that indicates if the user is part of the "Enterprise" plan.

There are the six dummy functions to validate all these fields. Here are the first three:

In [14]:
normalizeUsername :: String -> Maybe String
normalizeUsername uname
  | length uname < 3 = Nothing
  | all (\c -> isAlphaNum c || c == '_') uname = Just (map toLower uname)
  | otherwise = Nothing

checkUsernameAvailable :: String -> Maybe Bool
checkUsernameAvailable uname =
  -- In a real implementation, this would query a database
  case uname of
    "admin" -> Nothing
    "root" -> Nothing
    _ -> Just True

checkAndGeneratePasswordHash :: String -> String -> Maybe String
checkAndGeneratePasswordHash password uname
  | length password < 8 = Nothing
  | password == uname = Nothing
  | otherwise =
      if any isUpper password && any isLower password && any isDigit password
        then Just password -- In a real implementation, this should properly hash the password
        else Nothing

We're doing two things to reduce the noice around the point we're trying to make:

- One is that we use `Maybe` instead of `Either` to avoid the `String`s explaining the errors all over the code.
- The second one is that, in some cases, we would have to use `IO` effects. For example, to connect to a database. We also ignore this for now since it's irrelevant to what we want to learn.

Same as before, the actual implementations of these functions doesn't matter. What matters is how we use them.

Here are the other three:

In [15]:
generateUserID :: Maybe String
generateUserID =
  -- In a real implementation, this should generate a unique ID
  Just "123456"

checkForEnterprisePlan :: String -> Maybe Bool
checkForEnterprisePlan email
  | '@' `elem` email = Just $ dropWhile (/= '@') email == "@enterprise.com"
  | otherwise = Nothing

-- Simulate database insertion which could fail or return UserAccount if successful
insertUserIntoDatabase :: UserId -> String -> PasswordHash -> Email -> Bool -> Maybe UserAccount
insertUserIntoDatabase userID uname password email hasEnterpr =
  Just $ UserAccount userID uname email password hasEnterpr

Ok, we have all the building blocks we need to write our `registerUser` function. Same as before, let's see how it looks dealing with the effect manualy:

In [16]:
registerUser :: String -> String -> String -> Maybe UserAccount
registerUser uname password email =
  case normalizeUsername uname of
    Nothing -> Nothing
    Just normUname ->
      case checkUsernameAvailable normUname of
        Nothing -> Nothing
        Just _ ->
          case checkAndGeneratePasswordHash password normUname of
            Nothing -> Nothing
            Just passHash ->
              case generateUserID of
                Nothing -> Nothing
                Just userID ->
                  case checkForEnterprisePlan email of
                    Nothing -> Nothing
                    Just hasEnterp -> insertUserIntoDatabase userID normUname passHash email hasEnterp

Ok, so, what does the `registerUser` function do? Well:
- It first normalizes the username, 
- then checks if that normalized username is available, 
- then checks the password that the password is not week and generates the password hash if it's not, 
- then we generate a unique user id, 
- then check if the user is part of an enterprise plan, 
- and after all that, we insert the user into the database and return it.

This function looks a lot like `createUser` one. Pause the lesson and try to simplify it using Applicative operators same as we did with `createUser`, then come back and we'll do it together.

Pause the video. Create `undefined` definitions of all the helper functions but with the correct types, and simplify the `registerUser` function using Applicative Functors.

Ok. Could you do it? What issue did you encounter? If you actually tried, you migh've noticed a small but important difference between the `createUser` function that we could greatly simplify by using Applicative Functor and the `registerUser` function that we couldn't. And that is:

Unlike with `createUser` where all actions/effects are **independent** and the results are used all at once at the end, in `registerUser`, **we use the results** of some actions/effects **as inputs to other actions/effects**.

You can see, for example, that `checkUsernameAvailable` takes as input the result of running the `normalizeUsername` effect.

It turns out, using the result of one action as input to another action is supper common, but we don't have an abstraction to deal with this yet.

Don't worry, we just have to do the same thing we did to extract `map`, `Functor`, and `Applicative`, and all the abstractions we extracted so far:


We identify a repeating pattern, extract it, and use the abstraction instead of writting the whole pattern.

## Extracting the pattern for `Maybe`


So, what's the repeating pattern here:

In [17]:
registerUser :: String -> String -> String -> Maybe UserAccount
registerUser uname password email =
  case normalizeUsername uname of
    Nothing -> Nothing
    Just normUname ->
      case checkUsernameAvailable normUname of
        Nothing -> Nothing
        Just _ ->
          case checkAndGeneratePasswordHash password normUname of
            Nothing -> Nothing
            Just passHash ->
              case generateUserID of
                Nothing -> Nothing
                Just userID ->
                  case checkForEnterprisePlan email of
                    Nothing -> Nothing
                    Just hasEnterp -> insertUserIntoDatabase userID normUname passHash email hasEnterp

The case expressions are clearly the reppeating pattern, more specifically we could start by identifying this block of code:

```haskell
case normalizeUsername uname of
  Nothing -> Nothing
  Just normUname -> ...
```

Now, if we see the original funciton, the expression that we provide to the `case` expression always changes, and it's always of type `Maybe something`. Let's name the first "something" type `a`, so we can name the value `ma` for `Maybe a`:

```haskell
case ma of
  Nothing -> Nothing
  Just normUname -> ...
```

The `Nothing -> Nothing` part is always the same, so we leave it as is. And the `Just` part always starts with `Just` but the name of the result changes, so we can put a generic name to it like `val`, of type `a`, of course because it's the result of successfully valuating `ma`:

```haskell
case ma of
  Nothing -> Nothing
  Just val -> ...
```

Now we reached the three dots. That's where the pattern starts repeating again. This part is always a bit different, but there's one thing we know it's the same: The code that comes after uses `val` in some way, so we have to provide it as parameter. 

We can name the whole thing that comes after `cont` for "continuation" because it's what continues after our pattern, and provide `val` as parameter:

```haskell
case ma of
  Nothing -> Nothing
  Just val -> cont val
```

Awesome, we have our pattern extracted with generic names. Now we have to wrapp it in a function. We can use any name for the function. However, this code binds two `Maybe`s together by providing the result of the first maybe as parameter to the second maybe, so we can call it `bindMaybe`: 

```haskell
bindMaybe = case ma of
  Nothing -> Nothing
  Just val -> cont val
```

If we run this code, though, we'll get two errors because we don't have an `ma` and `cont` values, we're creating them out of thin air! The solution is the same as always, if our funtion needs more values, it can take them as parameters:

In [19]:
bindMaybe ma cont = case ma of
  Nothing -> Nothing
  Just val -> cont val

:t bindMaybe

bindMaybe :: forall t a. Maybe t -> (t -> Maybe a) -> Maybe a

And that's it! We extracted the pattern to handle the effect of running `ma` and providing the result to the rest of the code.

Now we can replace all instances of the pattern with this function:

In [20]:
registerUser' :: String -> String -> String -> Maybe UserAccount
registerUser' uname password email =
  bindMaybe (normalizeUsername uname)                         ( \normUname ->
  bindMaybe (checkUsernameAvailable normUname)                ( \_ ->
  bindMaybe (checkAndGeneratePasswordHash password normUname) ( \passHash ->
  bindMaybe generateUserID                                    ( \userID ->
  bindMaybe (checkForEnterprisePlan email)                    ( \hasEnterp ->
  insertUserIntoDatabase userID normUname passHash email hasEnterp)))))

This is significantly easier to read. You can see that, on each line, the first argument is the function that we're evaluating, and the second is the rest of the code. We use lambda functions at eacth step to to capture the resulting value as parameter and provide the rest of the code as body.

We can make it even more clear by infixing `bindMaybe`. That way, we have the action being evaluated to the left and the continuation to the right:

In [22]:
registerUser' :: String -> String -> String -> Maybe UserAccount
registerUser' uname password email =
  normalizeUsername uname                         `bindMaybe` ( \normUname ->
  checkUsernameAvailable normUname                `bindMaybe` ( \_ ->
  checkAndGeneratePasswordHash password normUname `bindMaybe` ( \passHash ->
  generateUserID                                  `bindMaybe` ( \userID ->
  checkForEnterprisePlan email                    `bindMaybe` ( \hasEnterp ->
  insertUserIntoDatabase userID normUname passHash email hasEnterp)))))


registerUser' "John"  "paSs123456" "john@enterprise.com"
registerUser' "Alice" "alice"      "alice@gmail.com"
registerUser' "root"  "root123456" "root@example.com"

Just (UserAccount {userId = "123456", userName = "john", userPassHash = "john@enterprise.com", userEmail = "paSs123456", userHasEnterprise = True})

Nothing

Nothing

And that's it! We're done extracting the pattern and replacing it in the code. We'll come back to this, but let's do a couple more first.

## Extracting the pattern for `Either e`

For completeness, let's do the same for a `registerUser` function that uses the `Either` type.

Let's start by redefining the helper functions. Here are the first three:

In [24]:
normalizeUsername :: String -> Either String String
normalizeUsername uname
  | length uname < 3 = Left "Username must be at least 3 characters"
  | all (\c -> isAlphaNum c || c == '_') uname = Right (map toLower uname)
  | otherwise = Left "Username can only contain alphanumeric characters and underscores"


checkUsernameAvailable :: String -> Either String Bool
checkUsernameAvailable uname =
  -- In a real implementation, this would query a database
  case uname of
    "admin" -> Left "Username 'admin' is reserved"
    "root" -> Left "Username 'root' is reserved"
    _ -> Right True

checkAndGeneratePasswordHash :: String -> String -> Either String String
checkAndGeneratePasswordHash password uname
  | length password < 8 = Left "Password must be at least 8 characters"
  | password == uname = Left "Password cannot be the same as username"
  | otherwise =
      if any isUpper password && any isLower password && any isDigit password
        then Right password -- In a real implementation, this should properly hash the password
        else Left "Password must contain uppercase, lowercase, and digit characters"

And here're the othe three: 

In [25]:
generateUserID :: Either String String
generateUserID =
  -- In a real implementation, this should generate a unique ID
  Right "123456"

checkForEnterprisePlan :: String -> Either String Bool
checkForEnterprisePlan email
  | '@' `elem` email = Right $ dropWhile (/= '@') email == "@enterprise.com"
  | otherwise = Left "Invalid email format"

-- Simulate database insertion which could fail or return UserAccount if successful
insertUserIntoDatabase :: UserId -> String -> PasswordHash -> Email -> Bool -> Either String UserAccount
insertUserIntoDatabase userID uname password email hasEnterpr =
  Right $ UserAccount userID uname email password hasEnterpr

Now that we finish with the boilerplate, this is the new `registerUser` with better errors:

In [26]:
registerUser :: String -> String -> String -> Either String UserAccount
registerUser uname password email =
  case normalizeUsername uname of
    Left err -> Left err
    Right normUname ->
      case checkUsernameAvailable normUname of
        Left err -> Left err
        Right _ ->
          case checkAndGeneratePasswordHash password normUname of
            Left err -> Left err
            Right passHash ->
              case generateUserID of
                Left err -> Left err
                Right userID ->
                  case checkForEnterprisePlan email of
                    Left err -> Left err
                    Right hasEnterp -> insertUserIntoDatabase userID normUname passHash email hasEnterp

Based on what you know now, pause the lesson and extract the pattern in a function called `bindEither`.

Hopefully, you arrived to something like this:

In [27]:
bindEither :: Either String a -> (a -> Either String b) -> Either String b
bindEither ea cont = case ea of
  Left err -> Left err
  Right val -> cont val

It's virtually the same as `bindMaybe` but with `Left err` instead of `Nothing` and `Right val` instead of `Just val`. Now, pause the lesson again and replace the pattern with the infix `bindEither` function.

Done? You should have something like this:

In [29]:
registerUser' :: String -> String -> String -> Either String UserAccount
registerUser' uname password email =
  normalizeUsername uname                         `bindEither` ( \normUname ->
  checkUsernameAvailable normUname                `bindEither` ( \_ ->
  checkAndGeneratePasswordHash password normUname `bindEither` ( \passHash ->
  generateUserID                                  `bindEither` ( \userID ->
  checkForEnterprisePlan email                    `bindEither` ( \hasEnterp ->
  insertUserIntoDatabase userID normUname passHash email hasEnterp)))))


registerUser' "John"  "paSs123456" "john@enterprise.com"
registerUser' "Alice" "alice"      "alice@gmail.com"
registerUser' "root"  "root123456" "root@example.com"

Right (UserAccount {userId = "123456", userName = "john", userPassHash = "john@enterprise.com", userEmail = "paSs123456", userHasEnterprise = True})

Left "Password must be at least 8 characters"

Left "Username 'root' is reserved"

So far so good. Lets do one a bit different.

## Extracting the pattern for `Log`

Let's say we have a datatype that contains the result of a computation plus a log that contains an explanation related to that computation. One possible implementation could be this one:

In [31]:
data Log a = Log [String] a deriving (Show, Eq)

add :: Int -> Int -> Log Int
add x y = Log ["Added " ++ show x ++ " and " ++ show y] (x + y)


add 4 6
add 3 7

Log ["Added 4 and 6"] 10

Log ["Added 3 and 7"] 10

In both cases the function returns ten. But the inputs are different. We wouldn't know the difference if it weren't for the logs. And whenever something breaks, you really want to have logs about it. I should point out that this is a highly innefficient and hard to use type, if you want to add logs to production code, don't use this.

Ok, for the sake of this lesson, lets change the original example to use `Log`. To avoid complicating things, we'll assume the functions can't fail anymore. You'll learn about how to combine effects in the future.

So, our first three helper functions now look like this:

In [32]:
import Data.Char (toLower)

normalizeUsername :: String -> Log String
normalizeUsername uname =
  if length uname < 3
  then Log ["Username too short: " ++ uname] (map toLower uname)
  else Log ["Username normalized: " ++ uname] (map toLower uname)

checkUsernameAvailable :: String -> Log Bool
checkUsernameAvailable uname =
  case uname of
    "admin" -> Log ["Username unavailable: " ++ uname] True
    "root" -> Log ["Username unavailable: " ++ uname] True
    _ -> Log ["Username is available: " ++ uname] True

checkAndGeneratePasswordHash :: String -> String -> Log String
checkAndGeneratePasswordHash password uname
  | length password < 8 = Log ["Password too short"] password
  | password == uname = Log ["Password matches username (insecure)"] password
  | otherwise = Log ["Password validated and hashed"] password

As you can see, the logic inside the functions never fails. Instead, it keeps going but logs what is wrong using the `String`. Of course, in a real implementations you'd have both the effect of logging information and failing. Same as the first two examples, what matters is not these definitions but what we do with them.

And here are other three:

In [33]:
generateUserID :: Log String
generateUserID = Log ["Generated user ID: 123456"] "123456"

checkForEnterprisePlan :: String -> Log Bool
checkForEnterprisePlan email =
  if '@' `elem` email
  then Log ["Checked enterprise status for: " ++ email] (takeWhile (/= '@') email == "@enterprise.com")
  else Log ["Invalid email format: " ++ email] False

insertUserIntoDatabase :: UserId -> String -> PasswordHash -> Email -> Bool -> Log UserAccount
insertUserIntoDatabase userID uname password email hasEnterpr =
  Log ["Inserted user into database: " ++ uname] $ UserAccount userID uname password email hasEnterpr

Now that we have the boilerplate out of the way, let's do the interesting part.

This is how we can handle the effect of logging by hand in our new `registerUser` function:

In [34]:
registerUser :: String -> String -> String -> Log UserAccount
registerUser uname password email =
  case normalizeUsername uname of
    Log logs1 normUname ->
      case checkUsernameAvailable normUname of
        Log logs2 _ ->
          case checkAndGeneratePasswordHash password normUname of
            Log logs3 passHash ->
              case generateUserID of
                Log logs4 userID ->
                  case checkForEnterprisePlan email of
                    Log logs5 hasEnterp ->
                      case insertUserIntoDatabase userID normUname passHash email hasEnterp of
                        Log logs6 account ->
                          Log (logs1 ++ logs2 ++ logs3 ++ logs4 ++ logs5 ++ logs6) account


Notice that the pattern is similar to the previous ones but with one important difference: The results are bindded between functions same as before, but the logs are accumulated at the end. That will affect a bit the implementation but the concept will be the same. 

Ok. So, we see that we repeat the `case` and `Log` lines 6 times an we have at the end an extra `Log` line. Let's copy the final 5 lines:

```haskell
bindLog = case checkForEnterprisePlan email of
            Log logs5 hasEnterp ->
              case insertUserIntoDatabase userID normUname passHash email hasEnterp of
                Log logs6 account ->
                  Log (logs1 ++ logs2 ++ logs3 ++ logs4 ++ logs5 ++ logs6) account

```

We just took the last two case statements at the end and named them `bindLog`. Why two case statements instead of just one as before? Well, because we need to specify how are we going to combine the logs, and we need two logs to do that. Try to derive this function with only one case expression and you'll see why we need two.

Now, let's generalize the firts case statement:

```haskell
bindLog = case l1 of
            Log logs1 val1 ->
              case insertUserIntoDatabase userID normUname passHash email hasEnterp of
                Log logs6 account ->
                  Log (logs1 ++ logs2 ++ logs3 ++ logs4 ++ logs5 ++ logs6) account

```

We can give te first `Log` effect the name `l1` which contains the `logs1` list of Strings and `val1` as result. 

Now, how should we call what continues next? right now it's a super-long function name with a bunch of parameters, but in reality, we can generalize this to a function that takes `val1` as parameter. We already did this before times, and we called it `cont` for continuation, so let's do that.

```haskell
bindLog = case l1 of
            Log logs1 val1 ->
              case cont val1 of
                Log logs6 account ->
                  Log (logs1 ++ logs2 ++ logs3 ++ logs4 ++ logs5 ++ logs6) account

```

The result of applying `cont` to `val1` and evaluating the `Log` effect generates a new `Log` value that contains the `logs2` list of `String`s and `val2` result, so let's do that:

```haskell
bindLog = case l1 of
            Log logs1 val1 ->
              case cont val1 of
                Log logs2 val2 ->
                  Log (logs1 ++ logs2 ++ logs3 ++ logs4 ++ logs5 ++ logs6) account
```

Awesome, so we "ran" two log effects, generating two `logs` values. The first effect returned the value `val1` that we provided to `cont` to generate the second value `val2`. Now, what we want is to make sure all our logs are present, so we'll concatenate them in the order whey were performed and return the final value:

```haskell
bindLog = case l1 of
            Log logs1 val1 ->
              case cont val1 of
                Log logs2 val2 ->
                  Log (logs1 ++ logs2) val2
```

Ok, we have the pattern here. But, same as before, we're creating `l1` and `cont` out of thin air, so let's provide them as arguments:

In [35]:
bindLog l1 cont = case l1 of
                     Log logs1 val1 ->
                       case cont val1 of
                         Log logs2 val2 ->
                           Log (logs1 ++ logs2) val2

:t bindLog

bindLog :: forall t a. Log t -> (t -> Log a) -> Log a

And we're done. We can explain the patterh by saying: "We take an action with the log effect and a continuation. We evaluate the action and provide the result to the continuation. Then, we evaluate the continuation to get the final result and return the final result with the logs combined in the correct sequence.

The only change in respect of the previous bindings is that we needed to extract a bigger pattern to be able to specify how to combine the logs. However, that's just an implementation detail. The key is that, by abstracting away the logging effect, we can replace the patterns with the `bindLog` function to get our final exression for `registerUser`:

In [37]:
registerUser' :: String -> String -> String -> Log UserAccount
registerUser' uname password email =
  normalizeUsername uname                         `bindLog` ( \normUname ->
  checkUsernameAvailable normUname                `bindLog` ( \_ ->
  checkAndGeneratePasswordHash password normUname `bindLog` ( \passHash ->
  generateUserID                                  `bindLog` ( \userID ->
  checkForEnterprisePlan email                    `bindLog` ( \hasEnterp ->
  insertUserIntoDatabase userID normUname passHash email hasEnterp)))))


registerUser' "John"  "pass123456" "john@enterprise.com"
registerUser' "Alice" "alice"      "alice@gmail.com"
registerUser' "root"  "root123456" "root@example.com"

Log ["Username normalized: John","Username is available: john","Password validated and hashed","Generated user ID: 123456","Checked enterprise status for: john@enterprise.com","Inserted user into database: john"] (UserAccount {userId = "123456", userName = "john", userPassHash = "pass123456", userEmail = "john@enterprise.com", userHasEnterprise = False})

Log ["Username normalized: Alice","Username is available: alice","Password too short","Generated user ID: 123456","Checked enterprise status for: alice@gmail.com","Inserted user into database: alice"] (UserAccount {userId = "123456", userName = "alice", userPassHash = "alice", userEmail = "alice@gmail.com", userHasEnterprise = False})

Log ["Username normalized: root","Username unavailable: root","Password validated and hashed","Generated user ID: 123456","Checked enterprise status for: root@example.com","Inserted user into database: root"] (UserAccount {userId = "123456", userName = "root", userPassHash = "root123456", userEmail = "root@example.com", userHasEnterprise = False})

This one was a bit different, and that's ok. How we define the bind function is highly dependent on the "effect" we want to represent. We did enough ground work to present our first definition for the Monad typeclass:

## Abstracting the `Monad` Type Class

If we zoom out a little bit, there are two things that we accomplished so far. One is that, if we take any of the `registerUser` definitions we had so far and change the name of the `bindSomething` function to just `bind`, we completely hid the effects and only left the sequence of logical steps visible:

```haskell
registerUser' uname password email =
  normalizeUsername uname                         `bind` ( \normUname ->
  checkUsernameAvailable normUname                `bind` ( \_ ->
  checkAndGeneratePasswordHash password normUname `bind` ( \passHash ->
  generateUserID                                  `bind` ( \userID ->
  checkForEnterprisePlan email                    `bind` ( \hasEnterp ->
  insertUserIntoDatabase userID normUname passHash email hasEnterp)))))
```

This is an amazing acomplishment! It means that the next person that reads the code will be able to focus on the logic of our function instead of dealing with the effects.

But on top of that, we generated three higher order functions that have the same interface and represent the same concept!:

```haskell
bindMaybe  :: Maybe    a -> (a -> Maybe    b) -> Maybe    b
bindEither :: Either e a -> (a -> Either e b) -> Either e b
bindLog    :: Log      a -> (a -> Log      b) -> Log      b
```

You can clearly see that the interface is the same. A function that takes an action of type `Something a` as a first parameter, runs the action to extract the value of type `a`, and applies the function provided as second parameter to this value of type `a` to get a new action of type `Something b`.

In other words, we're binding effects by allowing the second effect to use the result of the first effect. Something we didn't have an abstraction for before.

This seems (and it is!) a pretty useful concept, so we'll create a type class to represent it. Here's our initial approximation of the `Monad` typeclass definition:

**Initial approximation:**

"A `Monad` is a type with an operator to sequentially compose two actions, passing any value produced by the first as an argument to the second"

```haskell
class Monad m where
    (>>=) :: m a -> (a -> m b) -> m b
```

This is not the final type class. But it already encapsulates the concept of `Monad`, and it's enough to define a couple of instances so we can have something to work with while we figure out the rest.

Why do I use this weird symbol instead of writting something like `bindM` like the previous examples? Well, in case you didn't notice, on all three examples, it looked better to put the `bindSomehting` function as an infix function to clearly separate the first and second actions. So, the best way is to use this function is as an infix operator. Same as with `Applicative`'s "apply" (`<*>`) operator.

Using this definition, we could implement the `Monad` instances for `Maybe`, `Either e`, and `Log` like this:

```haskell
instance Monad Maybe where
    -- (>>=) :: Maybe a -> (a -> Maybe b) -> Maybe b
    (>>=) = bindMaybe

instance Monad (Either e) where
    -- (>>=) :: Either e a -> (a -> Either e b) -> Either e b
    (>>=) = bindEither

instance Monad Log where
    -- (>>=) :: Log a -> (a -> Log b) -> Log b
    (>>=) = bindLog
```

Which it may be too much indirection since you likely forgot the details of each `bindSomething` function, so let's replace the names with their definitions and simplify a bit to get the same instances:

```haskell
instance Monad Maybe where
    -- (>>=) :: Maybe a -> (a -> Maybe b) -> Maybe b
    (Just x) >>= k      = k x
    Nothing  >>= _      = Nothing

instance Monad (Either e) where
    -- (>>=) :: Either e a -> (a -> Either e b) -> Either e b
    Left  l >>= _ = Left l
    Right r >>= k = k r

instance Monad Log where
     -- (>>=) :: Log a -> (a -> Log b) -> Log b
    (Log logs val) >>= k =
      case k val of
        Log newLogs newVal -> Log (logs ++ newLogs) newVal
```

These are the same definitions we derived earlier but with a few esthetic changes. We pattern match instead of using case expressions and we changed the name of the continuation function from `cont` to `k` because that's the name we use as a convention. 

And with these instances, all definitions of `registerUser` would look like this:

```haskell
registerUser' uname password email =
  normalizeUsername uname                         >>= ( \normUname ->
  checkUsernameAvailable normUname                >>= ( \_ ->
  checkAndGeneratePasswordHash password normUname >>= ( \passHash ->
  generateUserID                                  >>= ( \userID ->
  checkForEnterprisePlan email                    >>= ( \hasEnterp ->
  insertUserIntoDatabase userID normUname passHash email hasEnterp)))))
```

## The abstraction `Monad`chy

To have a better understanding of the `Monad` type class, let's see how it relates to `Functor` and `Applicative`.

The relationship between `Functor`, `Applicative`, and `Monad` has a clear hierarchy both in theory and in practice. We won't discuss their relationship from the category theory point of view in this course, because you don't need to know that to be an effective Haskell developer. But what you do need to know to be an effective Haskell developer is how these type classes relate to each other and which one should you use when.

In the past, Phil Waddler discovered that the concept of `Monad` would be useful to sequence effects in our lazy language. We also knew that `Functor` was a very powerful concept. So, in the first Haskell report of '98, we had those two type classes but they were unrelated:

<img src="../images/haskell98_typeclasses.gif"
  style="
  display:block;
  margin-left: 20%;
  margin-right: auto;
  width: 64%;"/>

This diargam shows the hierarchy of Haskell classes defined in the Prelude and the Prelude types that are instances of these classes.

As you can see at the bottom, `Monad` didn't had any dependencies, `Functor` didn't relate to any other type class, and there was no `Applicative`. On top of that, the functions that these type classes provided were slightly different than the modern ones.

Even though the details of how this evolved to the current state and how it will still evolve are interesting, for the sake of time, thisis the overivew:

- 1998: "The Haskell 98 Report" is presented. It contains `Functor` typeclass with one behavior (`fmap`) and the `Monad` type class with four behaviors (`>>=`,`>>`, `return`, and `fail`).

- 2008: Conor McBride and Ross Paterson in their paper "Applicative programming with effects" present the `Applicative` type class, which filled a conceptual gap between `Functor` and `Monad` by enabling function application within a context/effect but without binding.

- 2014: "The Applicative-Monad Proposal (AMP)" was presented to reconcile the mathematical hierarchy—where every monad is an applicative functor—with the language’s implementation. The key motivation was to reduce code duplication, make Haskell more beginner friendly, and achieve theoretical consistency. Among other things, the AMP proposal added `Applicative` to the Prelude and made it a superclass of `Monad`.

- 2018: "The MonadFail Proposal (MFP)" was presented to remove the `fail` behavior from `Monad`. The problem with having this is that `fail` cannot be sensibly implemented for many monads, defaulting to `error` for those cases. As a consequence, you couldn't use `Monad` polymorphic code safely.

Ok, then, you might say, why should I care about the history of these type classes? Why not just telling me the current state and that's it? Well, because code doesn't magically update itself once we decide to change something. Things take time, and transitions that could potentially break thousands of projects and tens of thousands of libraries have to be done carefully. This, for you, the beginner, means that you might see things that don't make sense, assume there's a reason you don't understand, and become frustrated.

The reality is that, most of the things that confuse beginners stop being confusing after learning the history behind it. The `Monad` type class is an example. This is the current `Monad` type class as defined in the Prelude at the time of recording:

```haskell
class Applicative m => Monad m where
    -- Sequentially compose two actions, passing any value produced by the first as an argument to the second.
    (>>=) :: m a -> (a -> m b) -> m b

    -- Sequentially compose two actions, discarding any value produced by the first.
    -- Conceptually the same as *>
    (>>)  :: m a -> m b -> m b
    m >> k = m >>= \_ -> k 

    -- Inject a value into the monadic type.
    -- Exactly the same as pure
    return      :: a -> m a
    return      = pure 
```

As you can see, the `m` type variable in `Monad m` is constrained by the `Applicative` type class. This makes sense since in 2014 we added the requirement that every `Monad` is an `Applicative` functor.

Another thing that makes sense is the bind operator (`>>=`), since it's the function that represents the concept of the `Monad` type class. All good so far. However, what comes next could be confusing for newcommers.

You might remember the second opperator from the IO lesson. We call it the "then" operator (`>>`). This operator sequentially composes two actions, discsrding any value produced by the first. Wait, what? We already have an operator that does that using the `Applicative` type class (`*>`). The then operator is defined based on the bind operator, so it's implementation might be different, but, we're creating a new operator for the same concept!

To make things worst, we see that the last behavior, `return`, that we also remember from the IO lesson, not only represents the same concept as pure, lifting a pure value to the `Applicative` and now `Monadic` context, but on top of that, it uses `Applicative`'s `pure` as its the fault definition! Even the implementation is thesame!

What is going on here!?

Well, you guessed it, history is what's going on. Because we didn't have the `Applicative` type class when `Monad` was introduced, all behaviors related to lifting and sequencing were part of the `Monad` type class. And now that we do have the `Applicative` type class, we have a lot of code  and repetition.

I'm sad to say that this is not the only case like this, but we're working on it! In 2021, "The Monad of no return" proposal was presented. It aims to remove `return` and `>>` from the `Monad` type class. However, **this proposal is still in discussion to this day**. There's a lot of discussion about this subject, some like the proposal, others don't. Whatever is the case now or in the future, you now now the reason and use case of every behavior in the `Monad` type class, but there's one question we have to answer for the current time being, and that is:

Which type class should I use if more than one would do?

Due to the close relation between these type classes, sometimes you could achieve the same thing with more than one. For example, if you want to sequnce effects ignoring the values, you could use both `Applicative` or `Monad`. So, which one should you use?

Here's a diagram that might help:

<img src="../images/abstraction_pyramid.png"
  style="
  display:block;
  margin-left: 20%;
  margin-right: auto;
  width: 64%;"/>

This stack of cylinders clearly states the hierarchy between these type classes. All applicatives are functors, and al monads are applicatives, but not the other way arround. So, many types will be able to have a meningful functor instance that follows all functor's laws. A smaller subset of those types will be able to have a meningful applicative instance that follows all applicative's laws. And and even smaller subset will be able to have a mieaningful monad instance that follows all monadic laws. 

The more up you go, the more restrictive it is (because your type has to comply with stricter rules), but the more power that type will have in exchange.

In general, you want to restrict your implementations as little as possible, so, unless there's a reason not to, you should prefer the less restrictive type. If, further along the line, you realize you need more power, you can move up the ladder.

So, in other words, always use the least powerful abstraction that satisfies your needs. This will help you write more general and reusable code.

Now, before moving on into figuring out the `Monad` laws, let's do some quick mental exercises. What is the result of:

In [39]:
return 3 :: Maybe Int

Just 3

In [40]:
Right "Hey" >>= (\x -> Right (x ++ "!"))

Right "Hey!"

In [41]:
Left "Hey" >>= (\x -> Right (x ++ "!"))

Left "Hey"

In [42]:
unoReverse x = Log ["uno reverse"] (not x)

return True >>= unoReverse >>= unoReverse

: 

Ahaha! What happened here? `Maybe` and `Either`'s `Monad` instances come out of the box, but we just came up with the `Log` type, so we have to actually define the `Monad` instance. And because of what we just discussed, to define the `Monad` instance, we need the `Applicative` instance, which in turn needs the `Functor` instance. So, let's define them in order. This is the `Functor` instance for the `Log` type:

In [43]:
instance Functor Log where
  fmap f (Log logs val) = Log logs (f val)

---------------- Check laws ----------------
-- If it follows identity law, it implicitly follows the composition law)
fmap id (Log ["test"] 1) == id (Log ["test"] 1)

True

In the case of `Log`, the logs are part of the structure, so we only apply the function to the value and that's it. We also check that the instance follows the Functor laws. 

Now, we can define the `Applicative` instance like this:

In [44]:
instance Applicative Log where
  pure = Log []
  (Log logs1 f) <*> (Log logs2 val) = Log (logs1 ++ logs2) (f val)
  --(Log logs1 f) <*> (Log logs2 val) = Log logs2 (f val)

---------------- Check laws ----------------
valueLog = Log ["some value"] 6
timesTwoLog = Log ["times two"] (*2)
showLog = Log ["show it"] show

(pure id <*> valueLog) == valueLog -- Identity

(pure show <*> pure 3) == ((pure (show 3)) :: Log String) -- Homomorphism

((pure (.) <*> showLog <*> timesTwoLog) <*> valueLog) == (showLog <*> (timesTwoLog <*> valueLog)) -- Composition

(timesTwoLog <*> pure 3) == (pure (\f -> f 3) <*> timesTwoLog ) -- Interchange

True

True

True

True

For the `Applicative` instance, we said that the `Log` type represents the effect of logging. So, to define `pure` we need to produce a `Log` value with no effect. What would be to produce no logging effect? In our implementation, it would be providing a list of logs with no log inside it. No logs, no effect. That is the hard one. 

The apply operator is a bit easier. We just pattermatch to extract the logs and values of both structures. Then, because we know that the first value is a function that we need to apply to the second value, we do that. 

The value is taken care off. What about the effect? We could think of two obvious things:
- One is that we could keep the last one only, loosing all previous logs.
- The other is to concatenate them in the order we evaluated them. 

Any other choice wouldn't make sense.

So, between these two, the second choice would keep all the logs in the same order as the expressions we evaluated, which is the whole point of having logs, so, that's intuitively the best choice. And if you're not convinced it's the best choice, why not check with the laws?

If we check all four Applicative laws for the case of keeping only the last log, we see that our instance breaks the Interchange law. However, if we cocatenate the logs, which is the option that makes more sense, we'll see that we do respect all laws.

Great! Now we can define the `Monad` by providing the definition of binding like this:

In [45]:
instance Monad Log where
     -- (>>=) :: Log a -> (a -> Log b) -> Log b
    (Log logs val) >>= k =
      case k val of
        Log newLogs newVal -> Log (logs ++ newLogs) newVal

Ok, now let's try again the mental exercise that didn't work. What is the result of:

In [46]:
unoReverse x = Log ["uno reverse"] (not x)

return True >>= unoReverse >>= unoReverse

Log ["uno reverse","uno reverse"] True

As you can see, thanks to our logs, we know that the final `True` value was reversed two times before being returned. We have a working instance of `Monad` for a type we came up on our own. However, the `Monad` type class is not just the biding operator. 

Sadly, same as with `Semigroup`, `Monoid`, `Functor`, and `Applicative`, not everything is reflected in the type class definition. So, we have to complement it with a few laws.

## The `Monad` laws

Monads have three laws that we have to look out for when we create instances of the type class. 

The first one is the **left identity** law:

**Left identity**

```haskell
return a >>= k = k a
```

Remember that `return` is just a regular function that is equal to `pure` and `k` is just the continuation function. So, lifting a pure value to the `Monad` level and binding the result of that with the continuation function is equal to just applying the continuation function to the pure value directly. Let's see how this looks for all the instances we worked with sofar, starting with the `Maybe` type:

In [47]:
k1 x = Just (x + 1) -- Some computation that might fail
k2 _ = Nothing      -- Some computation that might fail

(return 3 >>= k1) == (k1 3) 
(return 3 >>= k2) == (k2 3) 

(return 3 >>= k1)

True

True

Just 4

We have two continuation functions, one where the function succeeds, and another where it fails. We can see that the left identity law is respected on both cases.

Trivially, the same can be seen for the `Eiter e` type:

In [48]:
k1 x = Right (x + 1)    -- Some operation that might fail with `Either e`
k2 x = Left "I failed"  -- Some operation that might fail with `Either e`

(return 3 >>= k1) == (k1 3) 
(return 3 >>= k2) == (k2 3) 

(return 3 >>= k2)

True

True

Left "I failed"

And finally, we can also check this law for our `Log` type:

In [49]:
k1 x = Log ["I added 1 to " ++ show x] (1+x) -- Some operation that logs

(return 3 >>= k1) == (k1 3) 
(return 3 >>= k1)

True

Log ["I added 1 to 3"] 4

We have a continuation function that takes any number and adds one to it. However, it also logs its action. Since `return` is just `pure` wich we defined it to be equal to an empty log (`Log [] x`), `return 3` is just `Log [] 3`. Then, if we run the effect of that with the binding operation, we get the pure three back and we concatenate the empty list with whatever string `k1` has. So, we end up with the same `String` `k1` had. So, the whole thing on the lefts side of the equals sign is the same as jut giving the pure `3` directly to the continuation funciton, since concatenating a list with an empty list is the same as doing nothing.

Ok, we have one down, two to go! The next one is also super easy:

**Right identity**
```haskell
    m >>= return = m
```

Binding an action with `return`, is the same as the original action. This is because the bind operation runs the effect and provides the result to `return` which in turn lifts the result back into the `Monad` level but without any real effect.

Let's see a few examples:

In [50]:
(Just 3 >>= return) == Just 3

(Left "error" >>= return) == Left "error"

(Log ["something"] 3 >>= return) == Log ["something"] 3

True

True

True

Between the left and rigth identity laws, we have proven that the `return` function can be used to lift values or functions to the `Monad` level on both sides of the bind operator without changing the effects. Using the terminology we used for previous lessons, the `return` function should respect the `Monad`'s structure.

The third, and last, law is the **Associativity law**:

**Associativity**
```haskell
    m >>= (\x -> k x >>= h) = (m >>= k) >>= h
```

The Associativity law, much like `Applicative`'s composition law, ensures that monadic composition is associative. 

Binding the first action with the result of binding the second and third actions is the same as binding the first and second actions and then binding third action. 

What this means in practice is that we can regroup actions and nest binds in a way that makes sense to us without changing the end result.

Let's check if our instances follow this law:

In [51]:
m = Just 5
k x = Just (x * 2)
--k x = Nothing
h x = Just (x + 3)

leftSide = m >>= (\x -> k x >>= h)
rightSide = (m >>= k) >>= h


leftSide == rightSide
leftSide

True

Just 13

We choose `Just 5` as the first action, then, the two actions that we'll bind will be one that multiplies by two, and then one that adds three. If this law holds, it means that we should get `5*2+3 = 13` independent on how we group the actions.

This law should also hold if any of the actions fail, we can check by changing the result of any of the actions to `Nothing`.

In both cases the law holds, we have a valid `Monad` instance for the `Maybe` type.

In [52]:
m = Right 5
k x = Right (x * 2)
--k x = Left "Error"
h x = Right (x + 3)

leftSide = m >>= (\x -> k x >>= h)
rightSide =(m >>= k) >>= h


leftSide == rightSide
leftSide

True

Right 13

Here we an see that the `Either e` type also follows the associativity law in both cases, we have a valid `Monad` instance for the `Either e` type. Not surprising, I know.

What about our custom `Log` type?:

In [53]:
m = Log ["initial"] 5
k x = Log [show x ++ " times two"] (x * 2)
h x = Log [show x ++ " plus three"] (x + 3)

leftSide = m >>= (\x -> k x >>= h)
rightSide =(m >>= k) >>= h


leftSide == rightSide
leftSide

True

Log ["initial","5 times two","10 plus three"] 13

Here we have the same action and functions but with `Log` effects. As you can see, each action has a different log. Based on what we know until now, would the law hold?

If we run this, we see that yes! It indeed holds. In both cases, the logs are in the correct order and the value is also `13`. We can also see in the logs the order the effects were run was correct. First `5*2`, and then the result of that,`10`, plus `3`. 

And those are the three `Monad` laws. Way easier than `Applicative` laws, right?

With this, we covered everything needed to show the `Monad` type class as defined in `base`

## `Monad` as defined in `base`

The `Monad` type class is currently defined in `base` as:

**A `Monad` is an `Applicative` with operations to embed pure expressions (`return`) and to sequentially compose two actions, both passing any value produced by the first as an argument to the second (`>>=`) and discarding any value produced by the first (`>>`)**

```haskell
class Applicative m => Monad m where
    (>>=) :: m a -> (a -> m b) -> m b

    (>>)  :: m a -> m b -> m b
    m >> k = m >>= \_ -> k

    return      :: a -> m a
    return      = pure


```
**That follows these laws:**

```haskell
return a >>= k = k a -- Left identity

m >>= return = m -- Right identity

m >>= (\x -> k x >>= h) = (m >>= k) >>= h -- Associativity
```

By the time you're watching this recording, this might or might not still be the definition in base. If "The Monad of no return" proposal we discussed earlier or a similar one is implemented, either or both of `return` and `>>` might be removed from the `Monad` type class. Which doesn't mean you shouldn't use them, since they'll still likely be left on the Prelude as helper functions, but it does hint at the one thing that is unique to the `Monad` class, the bind (`>>=`) operator.

And that's it! You know everything you need to know about the `Monad` type class. From now on, every time you see a type that is an instance of the `Monad` type class, the only think you have to figure out is the effect that it represents. If you know that, you don't even need to know how it's implemented under the hood because you already know the behaviors of all `Monad`s. The `IO` `Monad` is an example, we didn't explore how it works under the hood, but we can use its full power through the Monad interface. Which reminds me:

## You have to `do` what you have to `do`

If you recall lesson 10, we learned about basic IO and how Haskell provides the `do` notation as syntactic suggar for expressions composed by the bind (`>>=`) and then (`>>`) `IO` operators:

In [54]:
unscramble :: IO ()
unscramble = do
  putStrLn "Write a bunch of numbers and letters scrambled together:"
  arg <- getLine
  let numbers = filter (`elem` "1234567890") arg
      letters = filter (`notElem` numbers) arg
  putStrLn $ "Numbers: " ++ numbers
  putStrLn $ "Letters: " ++ letters


unscramble

Write a bunch of numbers and letters scrambled together:
Numbers: 564
Letters: akjd

Well, everything you learned in that lesson about `do` notation applies not only to the `IO` monad, but to every type that implements the `Monad` type class!! That means we cand do this, for example:

In [55]:
registerUser :: String -> String -> String -> Log UserAccount
registerUser uname password email = do
  normUname <- normalizeUsername uname
  _         <- checkUsernameAvailable normUname
  passHash  <- checkAndGeneratePasswordHash password normUname
  userID    <- generateUserID
  hasEnterp <- checkForEnterprisePlan email
  insertUserIntoDatabase userID normUname passHash email hasEnterp


registerUser "admin" "admin123" "person@enterprise.com"

Log ["Username normalized: admin","Username unavailable: admin","Password validated and hashed","Generated user ID: 123456","Checked enterprise status for: person@enterprise.com","Inserted user into database: admin"] (UserAccount {userId = "123456", userName = "admin", userPassHash = "admin123", userEmail = "person@enterprise.com", userHasEnterprise = False})

This is the same function we wrote before, but since we created an instance of the `Monad` class for `Log`, we can now use `do` notation.

We won't go into all the examples and details again, since you can go see lesson 10 for that and the lesson is already quite long. So, let's finish here for now.

## And that's it for today! 😃

We haven't talked about all the functions that you get for free when you create an instance of the `Monad` type class, and that's because your homework is to go to the `Control.Monad` module in the `base` library and explore them! Take your time to understand and use all of them. In some cases, you'll see that a few functions are also constrained by other type classes that we haven't discussed. So far during this course you have seen me go through over a dozen type classes, you know how to do it by yourself by now. I believe in you!

Make sure to do your homework, and I'll see you in the next one!